### Install

In [235]:
#pip install ucimlrepo

In [236]:
#pip install psycopg2

### Imports

In [237]:
# Imports
from ucimlrepo import fetch_ucirepo
import pandas as pd
import psycopg2
from collections import defaultdict
from scipy.stats import entropy
from scipy import stats as kl
from decimal import Decimal


In [247]:
connection = psycopg2.connect(database="Project645", user="vaishnavishah", password="10jan2001", host="localhost", port=5432)

cursor = connection.cursor()


In [239]:
# cursor.execute("SELECT * from unmarried;")
# record = cursor.fetchall()
# unmarried_df = pd.DataFrame(record, columns=['age', 'workclass', 'fnlwgt', 'education', 'education-num',
#        'occupation', 'relationship', 'race', 'sex',
#        'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])

In [240]:
# unmarried_df

In [248]:
# query = "select workclass, sum(age) from married where workclass is not null group by workclass order by workclass;"

# cursor.execute(query)
# record = cursor.fetchall()
# print(record)

[('Federal-gov', 32541), ('Local-gov', 68256), ('Private', 605510), ('Self-emp-inc', 58702), ('Self-emp-not-inc', 121574), ('State-gov', 38963), ('Without-pay', 801)]


In [241]:
# cursor.execute("SELECT * from married;")
# record = cursor.fetchall()
# married_df = pd.DataFrame(record, columns=col_names)

In [242]:
results_kl = defaultdict(list)

In [243]:
views = []
continuous=['age','fnlwgt', 'education_num','capital_gain','capital_loss','hours_per_week']
discrete=['workclass','education','occupation','relationship','race','sex','native_country']
aggregate=['sum','avg','max','min','count']

k=10
num_partitions=20
delta= 0.01

In [244]:
def get_all_views():
    views=[]
    for f in aggregate:
        for a in discrete:
            for m in continuous:
                views.append((a,m,f))
    return views

def get_query_result( conn, query ) :
    cur = conn.cursor()
    cur.execute( query )
    return cur

def normalize(f_list_1,f_list_2):
    sum_1=sum(f_list_1)
    sum_2=sum(f_list_2)
    norm_1 = [i/sum_1 for i in f_list_1]
    norm_2=[i/sum_2 for i in f_list_2]
    return norm_1,norm_2

def get_utility(f_list_1,f_list_2):
    nf_1,nf_2=normalize(f_list_1,f_list_2)
    nf_1=[float(x) for x in nf_1]
    nf_2=[float(x) for x in nf_2]
    return kl.entropy(nf_1,nf_2)


def get_top_k(dict_kl, k):
    return sorted(dict_kl.items(), key= lambda item: item[1], reverse=True)[:k]

def get_iter(d):
    if isinstance(d,dict):
        return d.items()
    else: #cursor
        return d.fetchall()

def get_agg_lists(o_1,o_2):
    dict_res = defaultdict(list)
    f_list_1 = []
    f_list_2 = []
    for k,v in get_iter(o_1):
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))
    for k,v in get_iter(o_2):
        if k not in dict_res:
            dict_res[k].append(Decimal(1e-10))
        if v != 0:
            dict_res[k].append(v)
        else:
            dict_res[k].append(Decimal(1e-10))
    for k,v in dict_res.items():
        if len(v) != 2:
            dict_res[k].append(Decimal(1e-10))
    for i in dict_res.values():
        f_list_1.append(i[0])
        f_list_2.append(i[1])
    return f_list_1,f_list_2


In [250]:
results_kl=defaultdict(list)

def get_all_views():
    views=[]
    for f in aggregate:
        for a in discrete:
            for m in continuous:
                views.append((a,m,f))
    return views

def get_query_result( conn, query ) :
    cur = conn.cursor()
    cur.execute( query )
    return cur

views=get_all_views()
for a,m,f in views:
    #print (a,f,m)
    q1='select '+a+', '+f+'('+m+') from married where '+a+' is not null group by '+a+' order by '+a+';'
    q2='select '+a+', '+f+'('+m+') from unmarried where '+a+' is not null group by '+a+' order by '+a+';'
    #print(q1)
    res_1=get_query_result(connection, q1)
    res_2=get_query_result(connection, q2)
    #print(res_1, res_2)
    print(res_1.fetchall())

[('Federal-gov', 32541), ('Local-gov', 68256), ('Private', 605510), ('Self-emp-inc', 58702), ('Self-emp-not-inc', 121574), ('State-gov', 38963), ('Without-pay', 801)]
[('Federal-gov', 133192704), ('Local-gov', 294034367), ('Private', 2788907935), ('Self-emp-inc', 219782260), ('Self-emp-not-inc', 445386130), ('State-gov', 163181655), ('Without-pay', 2496472)]
[('Federal-gov', 8077), ('Local-gov', 17158), ('Private', 147025), ('Self-emp-inc', 13909), ('Self-emp-not-inc', 26136), ('State-gov', 10428), ('Without-pay', 130)]
[('Federal-gov', 888115), ('Local-gov', 1631524), ('Private', 21478917), ('Self-emp-inc', 7170199), ('Self-emp-not-inc', 5021801), ('State-gov', 841294), ('Without-pay', 2414)]
[('Federal-gov', 84351), ('Local-gov', 209191), ('Private', 1655869), ('Self-emp-inc', 238544), ('Self-emp-not-inc', 313874), ('State-gov', 95209), ('Without-pay', 1887)]
[('Federal-gov', 30978), ('Local-gov', 65728), ('Private', 633302), ('Self-emp-inc', 60820), ('Self-emp-not-inc', 117639), ('S